In [27]:
# import libraries
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.feature_selection import SelectPercentile, f_regression


# Transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

# Datasets / synthetic data
from sklearn.datasets import load_diabetes, make_regression

np.random.seed(42)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV



In [2]:
url = "https://raw.githubusercontent.com/rhodes-byu/stat-486/main/data/OceanicFisheries/ocean_data.csv"
df = pd.read_csv(url)

print(df.head())

   T_degC   Salnty  O2ml_L  Depthm  Bottom_D  Wind_Spd  Dry_T  Wet_T  \
0  16.830  33.8510   5.560      65    1337.0      14.0   16.5   15.5   
1   9.262  33.8481   2.729     140    1202.0       5.0   15.0   13.0   
2  15.390  33.4260   5.990      30    3871.0      10.0   18.8   17.6   
3  14.540  32.9470   5.840      42    4018.0      14.0   16.9   16.1   
4   7.410  34.1810   1.000     300    4058.0      21.0   16.3   14.9   

                       Wea      Cloud_Typ                  Cloud_Amt  \
0                      NaN            NaN                        NaN   
1            Partly Cloudy        Stratus  1/10 or less but not zero   
2  Continuous blowing snow        Stratus                      10/10   
3  Continuous blowing snow  Stratocumulus                      10/10   
4            Partly Cloudy  Stratocumulus               7/10 to 8/10   

     Visibility  
0           NaN  
1  10km to 20km  
2  10km to 20km  
3   4km to 10km  
4   4km to 10km  


In [3]:
# Drop columns that are not needed for analysis
df = df.drop(columns=['Wea', 'Cloud_Typ', 'Cloud_Amt', 'Visibility'])
print(df.head())

   T_degC   Salnty  O2ml_L  Depthm  Bottom_D  Wind_Spd  Dry_T  Wet_T
0  16.830  33.8510   5.560      65    1337.0      14.0   16.5   15.5
1   9.262  33.8481   2.729     140    1202.0       5.0   15.0   13.0
2  15.390  33.4260   5.990      30    3871.0      10.0   18.8   17.6
3  14.540  32.9470   5.840      42    4018.0      14.0   16.9   16.1
4   7.410  34.1810   1.000     300    4058.0      21.0   16.3   14.9


In [5]:
# make a training and a test set
# Separate features and target
X = df.drop(columns=['T_degC'])
y = df['T_degC']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=307,
    test_size=0.25
)

print(X_train.shape, X_test.shape)

(5329, 7) (1777, 7)


In [ ]:
# Pipeline
pipe = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scale', StandardScaler()),
    ('model', LinearRegression())
])

# b fit pipeline to the training data
pipe.fit(X_train, y_train)

# c report the training MSE
y_train_pred = pipe.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)

print(train_mse)

# d report the test MSE
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)

print(test_mse)



1.535095636242088
1.75433086109485


In [10]:
# e... compare
ytest_var = np.var(y_test)

print(ytest_var)

test_mse / ytest_var


14.8162872136925


np.float64(0.11840556515896788)

In [ ]:
# 3. Feature Importance 

# 3a order features by the magnitude of their coefficients 

betas = pipe.named_steps['model'].coef_

# Extract feature names AFTER polynomial expansion
feature_names = pipe.named_steps['poly'].get_feature_names_out(X_train.columns)

# Create DataFrame of coefficients
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': betas
})

# Order by magnitude of coefficient
coef_df['abs_coef'] = coef_df['coefficient'].abs()
coef_df_sorted = coef_df.sort_values('abs_coef', ascending=False)

coef_df_sorted.head(10)

,feature,coefficient,abs_coef
9,Salnty Depthm,147.569766,147.569766
2,Depthm,-146.550613,146.550613
1,O2ml_L,-37.244657,37.244657
12,Salnty Dry_T,35.153778,35.153778
5,Dry_T,-34.383804,34.383804
8,Salnty O2ml_L,32.217666,32.217666
6,Wet_T,-11.009095,11.009095
13,Salnty Wet_T,8.948392,8.948392
7,Salnty^2,-4.503595,4.503595
3,Bottom_D,-3.967908,3.967908


In [ ]:
# 3b top 3 largest positive
top_positive = coef_df_sorted.sort_values('coefficient', ascending=False).head(3)
top_positive


,feature,coefficient,abs_coef
9,Salnty Depthm,147.569766,147.569766
12,Salnty Dry_T,35.153778,35.153778
8,Salnty O2ml_L,32.217666,32.217666


In [ ]:
# 3c negative
top_negative = coef_df_sorted.sort_values('coefficient').head(3)
top_negative


,feature,coefficient,abs_coef
2,Depthm,-146.550613,146.550613
1,O2ml_L,-37.244657,37.244657
5,Dry_T,-34.383804,34.383804


In [14]:
# 3d estimated y-intercept
intercept = pipe.named_steps['model'].intercept_
intercept


np.float64(11.607865265528682)

In [15]:
# 4 Hyperparameter Tuning with Grid Search 

# 4a KNN model instead of linear regression
pipe_knn = Pipeline([
    ('impute', SimpleImputer()),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('scale', StandardScaler()),
    ('model', KNeighborsRegressor())
])

In [ ]:
# 4b GridSearch CV setup
param_grid = {
    'impute__strategy': ['mean', 'median'],
    'poly__degree': [1, 2, 3],
    'model__n_neighbors': list(range(5, 101, 5)),
    'model__weights': ['uniform', 'distance']
}

gs = GridSearchCV(
    pipe_knn,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=10
)

In [17]:
# 4d time the grid search
import time

start_time = time.time()
gs.fit(X_train, y_train)
end_time = time.time()

fit_time = end_time - start_time
fit_time


83.12749290466309

In [18]:
# 4c best hyperparameters and the best MSE
best_params = gs.best_params_
best_mse = -gs.best_score_

best_params, best_mse


({'impute__strategy': 'median',
  'model__n_neighbors': 10,
  'model__weights': 'distance',
  'poly__degree': 3},
 np.float64(1.1942865460094276))

In [19]:
# 4e use pipeline to get predictions for the test data
best_model = gs.best_estimator_

y_test_pred_knn = best_model.predict(X_test)


In [20]:
# 4f report the test MSE
test_mse_knn = mean_squared_error(y_test, y_test_pred_knn)
test_mse_knn

1.2398437245052845

In [23]:
# 5. Randomized Grid Search
pipe_knn = Pipeline([
    ('impute', SimpleImputer()),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('scale', StandardScaler()),
    ('model', KNeighborsRegressor())
])

param_dist = {
    'impute__strategy': ['mean', 'median'],
    'poly__degree': [1, 2, 3],
    'model__n_neighbors': list(range(5, 101, 5)),
    'model__weights': ['uniform', 'distance']
}

# RANDOMIZED SEARCH
rs = RandomizedSearchCV(
    pipe_knn,
    param_distributions=param_dist,
    n_iter=40,
    scoring='neg_mean_squared_error',
    cv=10,
    random_state=307
)

In [24]:
start_time = time.time()
rs.fit(X_train, y_train)
end_time = time.time()

rs_time = end_time - start_time
rs_time

13.211056232452393

In [25]:
rs_best_params = rs.best_params_
rs_best_mse = -rs.best_score_

rs_best_params, rs_best_mse


({'poly__degree': 3,
  'model__weights': 'uniform',
  'model__n_neighbors': 10,
  'impute__strategy': 'mean'},
 np.float64(1.2832593203192033))

In [26]:
best_rs_model = rs.best_estimator_

y_test_pred_rs = best_rs_model.predict(X_test)

rs_test_mse = mean_squared_error(y_test, y_test_pred_rs)

rs_test_mse


1.3121145795554305

In [30]:
# 6. Advanced Pipeline

# need to get the full dataset again bc the other one didn't have some columns..
url = "https://raw.githubusercontent.com/rhodes-byu/stat-486/main/data/OceanicFisheries/ocean_data.csv"
df_full = pd.read_csv(url)

In [31]:
X = df_full.drop(columns=['T_degC'])
y = df_full['T_degC']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=307
)

#define
categorical_features = ['Wea', 'Cloud_Typ', 'Cloud_Amt', 'Visibility']
numeric_features = X_train.columns.difference(categorical_features)


numeric_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scale', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
    ('select', SelectPercentile(score_func=f_regression, percentile=50))
])

#combine using column transfromer
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

#full pipeline with KNN model
advanced_pipe = Pipeline([
    ('preprocess', preprocessor),
    ('model', KNeighborsRegressor(
        n_neighbors=20,
        weights='distance'
    ))
])

In [32]:
#6a fit the pipeline
advanced_pipe.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [33]:
# Training MSE
y_train_pred = advanced_pipe.predict(X_train)
train_mse_adv = mean_squared_error(y_train, y_train_pred)

# Test MSE
y_test_pred = advanced_pipe.predict(X_test)
test_mse_adv = mean_squared_error(y_test, y_test_pred)

train_mse_adv, test_mse_adv


(7.21400727093023e-13, 1.4623776711365377)

In [ ]:
#8 save model
import joblib

joblib.dump(gs.best_estimator_, 'lab-03-final-model.joblib', compress=3)

['lab-03-final-model.joblib']